In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

import xgboost as xgb

from datetime import datetime
import lightgbm as lgb
#import multiprocessing
#import gc


In [2]:
from sklearn.preprocessing import MinMaxScaler  
##smote 데이터 스케일링

In [3]:
#!pip3 install imblearn

In [4]:
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE ##SMOTE 오예~ 

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
import os
import gc
import sys
sys.path.append("../..")

In [7]:
os.getcwd()

'/home/biglake0987/ybigta_first_proj/EDA'

In [8]:
os.listdir()

['FinalPT_RFECV---1.ipynb',
 'FinalPT_RFECV-1_XGBoost(error).ipynb',
 'smote_train.csv',
 'test_transaction.csv',
 'rfecv_1_train.csv',
 'V_cols_test.csv',
 'train_transaction.csv',
 'V_cols_train.csv',
 'FinalPT_RFECV-----1.ipynb',
 '.ipynb_checkpoints',
 'FinalPT_RFECV-1.ipynb',
 'rfecv_1_test.csv',
 'jinoo_train.csv',
 'FinalPT_RFECV-1_XGBoost(x).ipynb',
 'FinalPT_RFECV-----2.ipynb',
 'test_identity.csv',
 'rfecv_2_train.csv',
 'train_identity.csv',
 'SMOTE.ipynb',
 'FinalPT_RFECV-----1(to_csv).ipynb',
 'sample_submission.csv',
 'sample.ipynb',
 'SMOTE-Copy2.ipynb',
 'to_csv(진우).ipynb',
 'rfecv_2_test.csv',
 'mintae1.ipynb',
 'SMOTE-2.ipynb',
 'jinoo_test.csv',
 'V_cols(20190814).ipynb',
 '.gitkeep',
 'smote_test.csv']

In [9]:
trans = pd.read_csv('train_transaction.csv', index_col='TransactionID')
test_trans = pd.read_csv('test_transaction.csv', index_col='TransactionID')

In [10]:
ident = pd.read_csv('train_identity.csv', index_col='TransactionID')
test_ident = pd.read_csv('test_identity.csv', index_col='TransactionID')

In [11]:
sample_sub = pd.read_csv('sample_submission.csv', index_col='TransactionID')

In [12]:
train = trans.merge(ident, how='left', left_index=True, right_index=True)
test = test_trans.merge(test_ident, how='left', left_index=True, right_index=True)

In [13]:
del trans, ident, test_trans, test_ident
gc.collect()

77

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [16]:
final_cols = ['isFraud','TransactionAmt','TransactionDT',
 'P_emaildomain',
 'R_emaildomain', "ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", 
"addr1", "addr2", "dist1", "dist2", "C1", "C2", "C3", "C4", "C5", "C6", "C7", "C8", "C9", "C10", 
"C11", "C12", "C13", "C14", "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", 
"D13", "D14", "D15", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "id_01", "id_02", 
"id_03", "id_04", "id_05", "id_06", "id_07", "id_08", "id_09", "id_10", "id_11", 
"id_12", "id_13", "id_14", "id_15", "id_16", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", "id_24", 
"id_25", "id_26", "id_27", "id_28", "id_29", "id_30", "id_31", "id_32", "id_33", "id_34", "id_35", 
"id_36", "id_37", "id_38", 
'V1','V2','V3','V4','V5','V6','V7','V9','V12','V13','V17','V18','V19',
'V20','V23','V24','V25','V26','V29','V30','V33',
 'V34','V35','V36','V37','V38','V39','V40','V42','V43','V44','V45','V47','V48','V49','V51','V52','V53','V54','V55',
 'V56','V57','V58','V59','V60','V61','V62','V63','V64','V66','V67','V69','V70','V71','V72','V73','V74','V75','V76',
 'V77','V78','V79','V80','V81','V82','V83','V85','V86','V87','V90','V91','V92','V93','V94', 'V95', 'V96', 'V97',
 'V98','V99','V100','V101','V102','V103','V105','V109','V115','V124','V125','V126','V127','V128','V129', 'V130','V131',
 'V132', 'V133', 'V134','V135','V136','V137','V139','V140','V143','V145','V146','V147','V149','V150','V151','V152',
 'V156','V158','V159','V160','V161','V162','V164','V165','V166',
 'V167','V168','V169','V170','V171','V172','V173','V174','V175','V176','V177','V178','V179','V180','V182','V187','V188',
 'V189','V192','V198','V199','V200','V201','V202','V203','V204','V205','V206','V207','V208','V209','V210',
 'V211', 'V212','V213','V214','V215','V216','V217','V218','V219','V220','V221','V222','V223','V224','V225','V226',
 'V227','V228','V229','V230','V231','V232','V233','V234','V236','V239','V243','V244','V245','V246','V248','V250',
 'V251','V256','V257',
 'V258','V259','V260','V261','V262','V263','V264','V265','V266','V267','V268','V270','V271','V272','V273','V274',
 'V275','V276','V277','V278','V279','V280','V281','V282','V283','V284','V285','V286','V287','V288','V289','V290',
 'V291','V292','V293','V294','V295',
 'V296','V297','V298','V299','V300','V301','V303','V304','V306','V307','V308','V309','V310','V311','V312',
 'V313','V314','V315','V316','V317','V318','V319','V320','V321','V322','V323','V324','V326','V329','V331',
 'V332','V333','V335','V336','V338','V339',  'DeviceType',
 'DeviceInfo']

In [17]:

del_cols = []
no_cols= []
for col in train.columns:

  if col not in final_cols:
    del_cols.append(col)
  else:
    no_cols.append(col)
print(len(no_cols));print(len(del_cols))
print(no_cols == final_cols)

353
80
False


In [18]:
print(len(del_cols))
del_cols

80


['V8',
 'V10',
 'V11',
 'V14',
 'V15',
 'V16',
 'V21',
 'V22',
 'V27',
 'V28',
 'V31',
 'V32',
 'V41',
 'V46',
 'V50',
 'V65',
 'V68',
 'V84',
 'V88',
 'V89',
 'V104',
 'V106',
 'V107',
 'V108',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V138',
 'V141',
 'V142',
 'V144',
 'V148',
 'V153',
 'V154',
 'V155',
 'V157',
 'V163',
 'V181',
 'V183',
 'V184',
 'V185',
 'V186',
 'V190',
 'V191',
 'V193',
 'V194',
 'V195',
 'V196',
 'V197',
 'V235',
 'V237',
 'V238',
 'V240',
 'V241',
 'V242',
 'V247',
 'V249',
 'V252',
 'V253',
 'V254',
 'V255',
 'V269',
 'V302',
 'V305',
 'V325',
 'V327',
 'V328',
 'V330',
 'V334',
 'V337']

In [19]:
train = train.drop(del_cols, axis=1)
test = test.drop(del_cols, axis=1)

In [20]:
for column in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2']:
    train[column +  '_count_full'] = \
        train[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))
    test[column +  '_count_full'] = \
        test[column].map(pd.concat([train[column], test[column]], ignore_index=True).value_counts(dropna=False))

In [21]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

In [22]:
train.columns

Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
       'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'card3_count_full', 'card4_count_full', 'card5_count_full',
       'card6_count_full', 'addr1_count_full', 'addr2_count_full',
       'TransactionAmt_to_mean_card1', 'TransactionAmt_to_mean_card4',
       'TransactionAmt_to_std_card1', 'TransactionAmt_to_std_card4'],
      dtype='object', length=365)

In [23]:
# 준형이형꺼 처리
#id_30

def os(df):
    if df == "-999":
        df = -999
    elif "Windows" in df:
        df = "Windows"
    elif "Mac" in df:
        df = "Mac"
    elif "iOS" in df:
        df = "ios"
    elif "Android" in df:
        df = "Android"
    else:
        df = "other"
        
    return df

train["id_30"].fillna("-999", inplace=True)
train["OS"] = train["id_30"].apply(os)

test["id_30"].fillna("-999", inplace=True)
test["OS"] = test["id_30"].apply(os)

#id_31 
# latest 여부 / 브라우저 col 생성
latest_browser_list = ["samsung browser 7.0",
                  "opera 53.0",
                  "mobile safari 10.0",
                  "google search application 49.0",
                  "firefox 60.0",
                  "edge 17.0",
                  "chrome 69.0", 
                  "chrome 67.0 for android",
                  "chrome 63.0 for android",
                  "chrome 63.0 for ios",
                  "chrome 64.0",
                  "chrome 64.0 for android",
                  "chrome 64.0 for ios",
                  "chrome 65.0",
                  "chrome 65.0 for android",
                  "chrome 65.0 for ios",
                  "chrome 66.0",
                  "chrome 66.0 for android",
                  "chrome 66.0 for ios"
                 ]

def latest_browser(df):
    if df == "-999":
        df = -999
    elif df in latest_browser_list:
        df = 1
    else:
        df = 0
    return df

def browser(df):
    if df == "-999":
        df = -999
    elif "chrome" in df:
        df = "chrome"
    elif "safari" in df:
        df = "safari"
    elif "firefox" in df:
        df = "firefox"
    elif "ie" in df:
        df = "ie"
    elif "samsung" in df:
        df = "samsumg"
    elif "Samsung" in df:
        df = "samsumg"
    return df 
    

train["id_31"].fillna("-999", inplace=True)
train["Latest_browser"] = train["id_31"].apply(latest_browser)
train["Browser"] = train["id_31"].apply(browser)

test["id_31"].fillna("-999", inplace=True)
test["Latest_browser"] = test["id_31"].apply(latest_browser)
test["Browser"] = test["id_31"].apply(browser)

In [24]:
y_train = train['isFraud'].copy()

In [25]:
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()

In [26]:
# Fill numeric columns' NaN values with mean
for c in X_train.columns:
    if X_train[c].dtype=='float16' or  X_train[c].dtype=='float32' or  X_train[c].dtype=='float64':
        X_train[c].fillna(X_train[c].mean())
        X_test[c].fillna(X_train[c].mean())

In [27]:

# Fill categorical columns' NaN values with placeholder (-999)
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [ ]:
# Label-Encode categorical columns
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))

In [40]:
# 모델설정
sm = SMOTE(ratio='auto', kind='regular')

# train데이터를 넣어 복제함
X_resampled, y_resampled = sm.fit_sample(X_train,list(y_train))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))

After OverSampling, the shape of train_X: (1139754, 350)
After OverSampling, the shape of train_y: (1139754, 350) 

After OverSampling, counts of label '1': 569877
After OverSampling, counts of label '0': 569877


In [41]:
train.to_csv('smote_train.csv')
test.to_csv('smote_test.csv')